In [56]:
!pip install torch transformers esm huggingface-hub tokenizers tiktoken
!pip uninstall tensorflow -y
TOKENIZERS_PARALLELISM = False

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.5 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [109]:
!pip show fsspec

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: fsspec
Version: 2023.6.0
Summary: File-system specification
Home-page: http://github.com/fsspec/filesystem_spec
Author: 
Author-email: 
License: BSD
Location: /opt/conda/lib/python3.11/site-packages
Requires: 
Required-by: dask, datasets, evaluate, huggingface_hub, jupyter_scheduler, lightning, PyAthena, pytorch-lightning, torch


In [32]:
!pip install --upgrade transformers accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
import os
import scipy
import sklearn
import esm

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [34]:
import pandas as pd
import numpy as np

In [35]:
import torch
from torch.utils.data import Dataset
from torch import nn
import math

In [51]:
import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction
from transformers import PreTrainedTokenizerFast
from transformers import AutoModel, AutoTokenizer

In [37]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient

In [38]:
from sklearn.metrics import r2_score, mean_squared_error

In [9]:
# Step 1: Set Persistent TORCH_HOME Directory Path
efs_model_path = "/home/sagemaker-user/user-default-efs/torch_hub"

# Step 2: Set TORCH_HOME Environment Variable to Ensure Persistent Storage
os.environ['TORCH_HOME'] = efs_model_path
if not os.path.exists(efs_model_path):
    os.makedirs(efs_model_path)

# Step 3: Set Device for Computation (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 4: Login to HuggingFace to Access ESM3 Weights
# Optional optimization: Use `HUGGINGFACE_HUB_TOKEN` env variable to bypass manual login
huggingface_token = os.getenv('HUGGINGFACE_HUB_TOKEN')
if huggingface_token:
    login(token=huggingface_token)
else:
    login()  # Will prompt user to enter their API key if no token is set

In [85]:
# Step 5: Load ESM3 Model from HuggingFace Hub
model: ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1").to(device)
model = model.to(torch.float32)  # Convert to float32 if needed

# Step 6: Save Model Weights to Persistent EFS Directory (optional, already stored via TORCH_HOME)
# model.save_pretrained(efs_model_path)

print("Model loaded and saved successfully!")

Model loaded and saved successfully!


In [112]:
class ProteinTokenizer:
    def __init__(self, vocab):
        """
        Initializes the ProteinTokenizer with a given vocabulary.

        Args:
            vocab (list): A list of strings representing the vocabulary, including special tokens and amino acids.
        """
        # Create mappings from token to id and vice versa
        self.token_to_id = {token: idx for idx, token in enumerate(vocab)}
        self.id_to_token = {idx: token for token, idx in self.token_to_id.items()}

    def tokenize(self, sequence):
        """
        Tokenizes a protein sequence into individual characters.

        Args:
            sequence (str): A string representing the protein sequence.

        Returns:
            list: A list of individual character tokens.
        """
        return [char for char in sequence]

    def convert_tokens_to_ids(self, tokens):
        """
        Converts a list of tokens to their corresponding token IDs.

        Args:
            tokens (list): A list of character tokens.

        Returns:
            list: A list of integer token IDs.
        """
        return [self.token_to_id.get(token, self.token_to_id['[UNK]']) for token in tokens]

    def encode(self, sequence, add_special_tokens=True):
        """
        Encodes a protein sequence into token IDs.

        Args:
            sequence (str): A string representing the protein sequence.
            add_special_tokens (bool): Whether to add special tokens like [CLS] and [SEP].

        Returns:
            list: A list of integer token IDs representing the input sequence.
        """
        tokens = self.tokenize(sequence)
        token_ids = self.convert_tokens_to_ids(tokens)

        # Optionally add special tokens like [CLS] at the beginning and [SEP] at the end
        if add_special_tokens:
            token_ids = [self.token_to_id['[CLS]']] + token_ids + [self.token_to_id['[SEP]']]

        return token_ids

    def decode(self, token_ids, skip_special_tokens=True):
        """
        Decodes a list of token IDs back into a protein sequence.

        Args:
            token_ids (list): A list of integer token IDs.
            skip_special_tokens (bool): Whether to skip special tokens during decoding.

        Returns:
            str: The decoded protein sequence.
        """
        tokens = [self.id_to_token[token_id] for token_id in token_ids]
        if skip_special_tokens:
            tokens = [token for token in tokens if token not in ['[CLS]', '[SEP]', '[PAD]', '[UNK]']]
        return ''.join(tokens)


# Define the vocabulary: special tokens and amino acids
special_tokens = ['[PAD]', '[CLS]', '[SEP]', '[UNK]']
amino_acids = [
    'A', 'C', 'D', 'E', 'F',
    'G', 'H', 'I', 'K', 'L',
    'M', 'N', 'P', 'Q', 'R',
    'S', 'T', 'V', 'W', 'Y'
]
vocab = special_tokens + amino_acids

# Initialize the custom tokenizer
tokenizer = ProteinTokenizer(vocab=vocab)

# Example usage
sequence = "GLVM"
encoded_sequence = tokenizer.encode(sequence)
print("Encoded Token IDs:", encoded_sequence)

decoded_sequence = tokenizer.decode(encoded_sequence)
print("Decoded Sequence:", decoded_sequence)

Encoded Token IDs: [1, 9, 13, 21, 14, 2]
Decoded Sequence: GLVM


In [113]:
import os

efs_model_path = "/home/sagemaker-user/user-default-efs/torch_hub"

# Check if the directory exists
if os.path.exists(efs_model_path):
    print(f"The directory exists: {efs_model_path}")
else:
    print(f"The directory does not exist: {efs_model_path}")

# List files and directories in the specified path
files = os.listdir(efs_model_path)
print(f"Contents of {efs_model_path}:")
for f in files:
    print(f)

The directory exists: /home/sagemaker-user/user-default-efs/torch_hub
Contents of /home/sagemaker-user/user-default-efs/torch_hub:
.ipynb_checkpoints
results_fold
logs


In [114]:
class PDB_Dataset(Dataset):
    """
    A PyTorch Dataset class for handling protein sequences and contact numbers.
    """
    def __init__(self, df, tokenizer, label_type='regression', max_length=1022):
        """
        Initializes the PDB_Dataset.
        
        Args:
            df (pandas.DataFrame): DataFrame with two columns:
                - Column 0: Protein sequences (str or list).
                - Column 1: Contact number values (list).
            tokenizer (ProteinTokenizer): Instance of the custom ProteinTokenizer.
            label_type (str): 'regression' or 'binary'.
            max_length (int): Maximum number of amino acids per sequence.
        """
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.label_type = label_type
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        """
        Retrieves the tokenized sequence and corresponding labels.
        
        Args:
            idx (int): Index of the data point.
        
        Returns:
            dict: Dictionary containing 'sequence_tokens', 'attention_mask', and 'labels'.
        """
        item = {}
        
        # Retrieve protein sequence
        sequence = self.df.iloc[idx, 0]
        if isinstance(sequence, list):
            sequence = ''.join(sequence)
        sequence = sequence.upper()  # Ensure uppercase letters

        # Encode the sequence using the tokenizer
        encoded = self.tokenizer.encode(sequence, add_special_tokens=True)
        
        # Truncate or pad the tokenized sequence
        sequence_tokens = encoded
        attention_mask = [1] * len(sequence_tokens)

        # Define total length (including special tokens)
        total_length = self.max_length + 2  # [CLS] and [SEP]
        
        if len(sequence_tokens) < total_length:
            pad_length = total_length - len(sequence_tokens)
            sequence_tokens += [self.tokenizer.token_to_id['[PAD]']] * pad_length
            attention_mask += [0] * pad_length
        else:
            sequence_tokens = sequence_tokens[:total_length]
            attention_mask = attention_mask[:total_length]
        
        # Convert to tensors
        item['sequence_tokens'] = torch.tensor(sequence_tokens, dtype=torch.long)
        item['attention_mask'] = torch.tensor(attention_mask, dtype=torch.long)
        
        # Retrieve labels
        labels = self.df.iloc[idx, 1]
        
        # Truncate or pad labels to match max_length
        if len(labels) > self.max_length:
            labels = labels[:self.max_length]
        elif len(labels) < self.max_length:
            pad_label = -100  # Commonly used to ignore in loss computation
            labels += [pad_label] * (self.max_length - len(labels))
        
        # Convert labels to tensor
        if self.label_type == 'regression':
            labels = torch.tensor(labels, dtype=torch.float32)
        elif self.label_type == 'binary':
            labels = torch.tensor(labels, dtype=torch.float32)
        else:
            raise ValueError(f"Unsupported label type: {self.label_type}")
        
        item['labels'] = labels
        
        return item


In [115]:
class ESM3ForTokenClassification(nn.Module):
    """
    A custom model for token classification using ESM3.
    """
    def __init__(self, model, num_labels=1):
        """
        Initializes the ESM3ForTokenClassification model.
        
        Args:
            model (ESM3InferenceClient): Pretrained ESM3 model instance.
            num_labels (int): Number of labels for classification.
        """
        super(ESM3ForTokenClassification, self).__init__()
        self.esm3_model = model
        self.num_labels = num_labels
        
        # Determine hidden size from ESM3 model
        hidden_size = 1536
        
        # Classification layer
        self.classifier = nn.Linear(hidden_size, self.num_labels)
    
    def forward(self, sequence_tokens, attention_mask=None, labels=None):
        """
        Forward pass for token classification.

        Args:
            sequence_tokens (torch.Tensor): The amino acid tokens.
            attention_mask (torch.Tensor, optional): Attention masks.
            labels (torch.Tensor, optional): Labels for classification.

        Returns:
            SequenceClassifierOutput: Contains loss and logits.
        """
        # Pass the sequence tokens (instead of input_ids) to the ESM3 model
        outputs = self.esm3_model(sequence_tokens=sequence_tokens)  # Assuming sequence_tokens are used

        # Extract hidden states (adjust attribute based on ESM3's output)
        if hasattr(outputs, 'last_hidden_state'):
            hidden_states = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)
        else:
            hidden_states = outputs[0]  # Shape: (batch_size, seq_len, hidden_size)

        # Pass hidden states through the classifier
        logits = self.classifier(hidden_states)  # Shape: (batch_size, seq_len, num_labels)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                # Regression
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            elif self.num_labels == 2:
                # Binary classification (using BCEWithLogitsLoss)
                loss_fct = nn.BCEWithLogitsLoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                # Multi-class classification
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits
        )


In [116]:
def model_init_1():
    return ESM3ForTokenClassification(model=model, num_labels=1).to(device)

In [117]:
# Custom Loss Function for Masked Regression
class MaskedMSELoss(torch.nn.Module):
    def __init__(self):
        super(MaskedMSELoss, self).__init__()

    def forward(self, inputs, target, mask):    
        diff2 = (torch.flatten(inputs[:, :, 0]) - torch.flatten(target)) ** 2.0 * torch.flatten(mask)
        result = torch.sum(diff2) / torch.sum(mask)
        if torch.sum(mask) == 0:
            return torch.sum(diff2)
        else:
            return result

In [118]:
# Custom Masked Regress Trainer
class MaskedRegressTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        labels = labels.squeeze().detach().cpu().numpy().tolist()
        labels = [math.log(t + 1) if t != -100 else -100 for t in labels]
        labels = torch.unsqueeze(torch.FloatTensor(labels), 0)
        masks = ~torch.eq(labels, -100)

        # Run the model on the input tokens
        outputs = model(**inputs)
        logits = outputs.logits

        # Use a custom loss function
        loss_fn = MaskedMSELoss()
        loss = loss_fn(logits, labels, masks)

        return (loss, outputs) if return_outputs else loss

In [119]:
# Metrics Calculation Function
def compute_metrics_regr(eval_pred: EvalPrediction):
    preds = eval_pred.predictions[:, :, 0]  # Assuming num_labels=1
    labels = eval_pred.label_ids
    batch_size, seq_len = preds.shape
    out_labels, out_preds = [], []

    for i in range(batch_size):
        for j in range(seq_len):
            if labels[i, j] > -1:  # Assuming -100 is used for masked tokens
                out_labels.append(labels[i, j])
                out_preds.append(preds[i, j])

    # Compute metrics
    pearson_r = scipy.stats.pearsonr(out_labels, out_preds)[0]
    mse = mean_squared_error(out_labels, out_preds)
    r2 = r2_score(out_labels, out_preds)

    return {
        "pearson_r": pearson_r,
        "mse": mse,
        "r2_score": r2
    }

In [120]:
# Data Collator Function for Batch Processing
def collator_fn(batch):
    if len(batch) == 1:
        return batch[0]
    return batch

In [121]:
# Load and process the training set
train_set = pd.read_csv('./data/sema_2.0/train_set.csv')
train_set = train_set.groupby('pdb_id_chain').agg({
    'resi_pos': list,
    'resi_aa': list,
    'contact_number': list
}).reset_index()

# Create training dataset
train_ds = PDB_Dataset(
    df=train_set[['resi_aa', 'contact_number']],
    tokenizer=tokenizer,
    label_type='regression',
    max_length=1022  # Adjust as needed
)

# Load and process the test set
test_set = pd.read_csv('./data/sema_2.0/test_set.csv')
test_set = test_set.groupby('pdb_id_chain').agg({
    'resi_pos': list,
    'resi_aa': list,
    'contact_number_binary': list
}).reset_index()

# Create test dataset
test_ds = PDB_Dataset(
    df=test_set[['resi_aa', 'contact_number_binary']],
    tokenizer=tokenizer,
    label_type='binary',  # Changed to 'binary' as per the test set
    max_length=1022  # Ensure consistency with training set
)


In [122]:
# Step 10: Define Training Arguments
training_args = TrainingArguments(
    output_dir=os.path.join(efs_model_path, 'results_fold'),           # Output directory
    num_train_epochs=1,                    # Total number of training epochs
    per_device_train_batch_size=1,         # Batch size per device during training
    per_device_eval_batch_size=1,          # Batch size for evaluation
    warmup_steps=0,                        # Number of warmup steps for learning rate scheduler
    learning_rate=1e-05,                   # Learning rate
    weight_decay=0.0,                      # Strength of weight decay
    logging_dir=os.path.join(efs_model_path, 'logs'),                  # Directory for storing logs
    logging_steps=200,                     # Log every 200 steps
    save_strategy="no",                    # Do not save checkpoints
    do_train=True,                         # Perform training
    do_eval=True,                          # Perform evaluation
    eval_strategy="epoch",           # Evaluate after each epoch
    gradient_accumulation_steps=1,         # Number of steps before backpropagation
    fp16=False,                            # Use mixed precision
    run_name="PDB_regression",             # Experiment name
    seed=42,                               # Seed for reproducibility
    load_best_model_at_end=False,
    metric_for_best_model="r2_score",
    greater_is_better=True,
    # Removed 'use_cpu' as it's not a valid argument in TrainingArguments
)

In [129]:
def dummy_forward_check(model_instance, tokenizer, sequence="GLVM", max_length=1024):
    """
    Performs a dummy forward pass to verify model and tokenizer functionality.
    
    Args:
        model_instance (ESM3ForTokenClassification): The custom model instance.
        tokenizer (ProteinTokenizer): The tokenizer to encode the sequence.
        sequence (str): The protein sequence to test.
        max_length (int): Maximum sequence length.
    
    Returns:
        None
    """
    # Encode the sequence into token IDs using the tokenizer
    sequence_tokens = tokenizer.encode(sequence)

    # Define total length (including special tokens like [CLS] and [SEP])
    total_length = max_length + 2
    if len(sequence_tokens) < total_length:
        pad_length = total_length - len(sequence_tokens)
        sequence_tokens += [tokenizer.token_to_id['[PAD]']] * pad_length
    else:
        sequence_tokens = sequence_tokens[:total_length]

    # Create attention mask
    attention_mask = [1 if token_id != tokenizer.token_to_id['[PAD]'] else 0 for token_id in sequence_tokens]

    # Convert to tensors
    sequence_tokens = torch.tensor([sequence_tokens], dtype=torch.long).to(device)  # Shape: (1, seq_len)
    attention_mask = torch.tensor([attention_mask], dtype=torch.long).to(device)    # Shape: (1, seq_len)

    # Create dummy labels (for the sake of testing)
    labels = torch.tensor([[0.0] * max_length], dtype=torch.float32).to(device)

    # Forward pass with the ESM3 model
    outputs = model_instance(sequence_tokens=sequence_tokens, attention_mask=attention_mask, labels=labels)

    # Inspect the outputs
    print("Type of outputs:", type(outputs))
    print("Attributes of outputs:", dir(outputs))
    print("Contents of outputs:", outputs)

    # Add dummy checks
    assert hasattr(outputs, 'logits'), "Output does not contain logits."

    # Expected logits shape: (batch_size, seq_len, num_labels)
    expected_shape = (1, total_length, model_instance.num_labels)
    assert outputs.logits.shape == expected_shape, f"Unexpected logits shape: {outputs.logits.shape}. Expected: {expected_shape}"

    print("Dummy checks passed successfully!")

In [130]:
# Call the dummy check function
dummy_forward_check(model_init_1(), tokenizer, sequence="GLVM", max_length=256) #1022

OutOfMemoryError: CUDA out of memory. Tried to allocate 66.00 MiB. GPU 

In [23]:
# Step 11: Instantiate the Trainer
trainer = Trainer(
    model=model_init_1(),   # Use the custom ESM3-based model for token classification
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=collator_fn,
    compute_metrics=compute_metrics_regr
)

# Step 12: Train the Model
trainer.train()

# Step 13: Save the Fine-Tuned Weights
fine_tuned_model_path = os.path.join(efs_model_path, "fine_tuned_sema_3_ESM3.pth")
torch.save(trainer.model.state_dict(), fine_tuned_model_path)
print(f"Fine-tuned model saved at {fine_tuned_model_path}")

Sequence tokenizer found.
Encoded tokens (dummy): [6, 4, 7, 20]
Embedding dimension: 1536


NotAnAttrsClassError: <class 'str'> is not an attrs-decorated class.